<a href="https://colab.research.google.com/github/xxpsynagure/data-analytics/blob/main/Pragnya_4SO19CS110_BDA_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BDA Lab Assignment 3

## 1. MapReduce program in Java to find occurrences of each word in a text file.

### Step 1: Set up the environment and Start the Hadoop Server

In [ ]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz

In [ ]:
!tar -xzvf hadoop-3.3.4.tar.gz

In [ ]:
!cp -r hadoop-3.3.4/ /usr/local/

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"

In [ ]:
!/usr/local/hadoop-3.3.4/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in slave mode
hosts filename                   list of hosts to use in slave mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the
    

### Step 2: Write Mapper and Reducer Program in Java and create a jar file


In [ ]:
!cat /content/Programs/mapper.java

package com.wordcount.wc;
import java.io.IOException;
import java.util.StringTokenizer;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.io.LongWritable;

public class WordCountMapper extends Mapper <LongWritable, Text, Text, IntWritable>
{
    private Text wordToken = new Text();
    public void map(LongWritable key, Text value, Context context) throws IOException,
    InterruptedException
    {
        StringTokenizer tokens = new StringTokenizer(value.toString()); //Dividing String into tokens
        while (tokens.hasMoreTokens())
        {
            wordToken.set(tokens.nextToken());
            context.write(wordToken, new IntWritable(1));
        }
    }
}



In [ ]:
!cat /content/Programs/reducer.java

package com.wordcount.wc;
import java.io.IOException;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;
public class WordCountReducer extends Reducer <Text, IntWritable, Text, IntWritable>
{
    private IntWritable count = new IntWritable();
    public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException
    {
// gurukul [1 1 1 1 1 1....]
    int valueSum = 0;
    for (IntWritable val : values)
    {
        valueSum += val.get();
    }
    count.set(valueSum);
    context.write(key, count);
    }
}


In [ ]:
!cat /content/Programs/driver.java

package com.wordcount.wc;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.util.GenericOptionsParser;

public class WordCount
{
    public static void main(String[] args) throws Exception
    {
        Configuration conf = new Configuration();
        String[] pathArgs = new GenericOptionsParser(conf, args).getRemainingArgs();
        if (pathArgs.length < 2)
        {
            System.err.println("MR Project Usage: wordcount <input-path> [...] <output-path>");
            System.exit(2);
        }
        Job wcJob = Job.getInstance(conf, "MapReduce WordCount");
        wcJob.setJarByClass(WordCount.class);
        wcJob.setMapperClass(WordCountMapper.class);
        wcJob.setCombinerClass(Wo

### Step 3: Create an input text file for counting

In [ ]:
!mkdir ~/input

In [ ]:
!cat /content/input/input.txt

Good Morning
Good Afternoon
Good Evening
Good Night

### Step 4: Run the command to perform Map Reduce using WordCount.jar file




In [ ]:
!/usr/local/hadoop-3.3.4/bin/hadoop jar /content/WordCount.jar /content/input/input.txt /content/output

2022-10-11 18:48:41,027 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-11 18:48:41,161 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-11 18:48:41,161 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-11 18:48:41,318 INFO input.FileInputFormat: Total input files to process : 1
2022-10-11 18:48:41,348 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-11 18:48:41,900 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1831973220_0001
2022-10-11 18:48:41,900 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-11 18:48:42,111 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2022-10-11 18:48:42,112 INFO mapreduce.Job: Running job: job_local1831973220_0001
2022-10-11 18:48:42,121 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2022-10-11 18:48:42,131 INFO output.FileOutputCommitter: File Output Committer Algorithm version is 2
2022-1

### Output

In [ ]:
!cat /content/output/part-r-00000

Afternoon	1
Evening	1
Good	4
Morning	1
Night	1


## 2.  MapReduce program in Java to find sum of products sold in each country.


### Step 1: Download the dataset needed

### Step 2: Write Map Reduce Programs in Java to create the jar file


In [ ]:
!cat /content/Programs/mapper1.java

package com.productsum.wc;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.*;
import org.apache.hadoop.mapred.*;
public class SalesCountryDriver {
public static void main(String[] args) {
    // Create a configuration object for the job
    JobClient my_client = new JobClient();
    JobConf job_conf = new JobConf(SalesCountryDriver.class);
    // Set a name of the Job
    job_conf.setJobName("SalePerCountry");
    // Specify data type of output key and value
    job_conf.setOutputKeyClass(Text.class);
    job_conf.setOutputValueClass(IntWritable.class);
    // Specify names of Mapper and Reducer Class
    job_conf.setMapperClass(SalesMapper.class);
    job_conf.setReducerClass(SalesCountryReducer.class);
    // Specify formats of the data type of Input and output
    job_conf.setInputFormat(TextInputFormat.class);
    job_conf.setOutputFormat(TextOutputFormat.class);
    FileInputFormat.setInputPaths(job_conf, new Path(args[0]));
    FileOutputFormat.setOutputPath(job_con

In [ ]:
!cat /content/Programs/reducer1.java

package com.productsum.wc;
import java.io.IOException;
import java.util.*;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapred.*;
public class SalesCountryReducer extends MapReduceBase implements Reducer<Text, IntWritable, Text, IntWritable> {
    public void reduce(Text t_key, Iterator<IntWritable> values,
    OutputCollector<Text, IntWritable> output, Reporter reporter)
    throws IOException {
        Text key = t_key;
        int frequencyForCountry = 0;
        while (values.hasNext()) {
            IntWritable value = (IntWritable) values.next();
            frequencyForCountry += value.get();
        }
        output.collect(key, new IntWritable(frequencyForCountry));
    }
}


In [ ]:
!cat /content/Programs/driver1.java

package com.productsum.wc;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.*;
import org.apache.hadoop.mapred.*;
public class SalesCountryDriver {
    public static void main(String[] args) {
        JobClient my_client = new JobClient();
        // Create a configuration object for the job
        JobConf job_conf = new JobConf(SalesCountryDriver.class);
        // Set a name of the Job
        job_conf.setJobName("SalePerCountry");
        // Specify data type of output key and value
        job_conf.setOutputKeyClass(Text.class);
        job_conf.setOutputValueClass(IntWritable.class);
        // Specify names of Mapper and Reducer Class
        job_conf.setMapperClass(SalesMapper.class);
        job_conf.setReducerClass(SalesCountryReducer.class);
        // Specify formats of the data type of Input and output
        job_conf.setInputFormat(TextInputFormat.class);
        job_conf.setOutputFormat(TextOutputFormat.class);
        FileInputFormat.setInputPaths(job_conf

### Step 3: Execution

In [ ]:
!/usr/local/hadoop-3.3.4/bin/hadoop jar /content/ProductSum.jar /content/input/SalesJan2009.csv /content/output1

2022-10-11 19:34:32,028 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-11 19:34:32,183 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-11 19:34:32,183 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-11 19:34:32,206 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-10-11 19:34:32,285 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2022-10-11 19:34:32,358 INFO mapred.FileInputFormat: Total input files to process : 1
2022-10-11 19:34:32,381 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-11 19:34:32,631 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local182147993_0001
2022-10-11 19:34:32,631 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-11 19:34:32,830 INFO mapreduce.Job: The url to track the job: 

In [ ]:
!cat /content/output1/part-00000

Argentina	1
Australia	38
Austria	7
Bahrain	1
Belgium	8
Bermuda	1
Brazil	5
Bulgaria	1
CO	1
Canada	76
Cayman Isls	1
China	1
Costa Rica	1
Country	1
Czech Republic	3
Denmark	15
Dominican Republic	1
Finland	2
France	27
Germany	25
Greece	1
Guatemala	1
Hong Kong	1
Hungary	3
Iceland	1
India	2
Ireland	49
Israel	1
Italy	15
Japan	2
Jersey	1
Kuwait	1
Latvia	1
Luxembourg	1
Malaysia	1
Malta	2
Mauritius	1
Moldova	1
Monaco	2
Netherlands	22
New Zealand	6
Norway	16
Philippines	2
Poland	2
Romania	1
Russia	1
South Africa	5
South Korea	1
Spain	12
Sweden	13
Switzerland	36
Thailand	2
The Bahamas	2
Turkey	6
Ukraine	1
United Arab Emirates	6
United Kingdom	100
United States	462


## 3. MapReduce program in Java to find average sales ( AVG(price) ) per country.


### Step 1: Download the dataset needed

### Step 2: Write Map Reduce Programs in Java to create the jar file

In [ ]:
!cat /content/Programs/mapper2.java

package com.avgsales.wc;
import java.io.IOException;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
public class Average_sales {
    public static class MapperClass extends Mapper<LongWritable, Text, Text, FloatWritable> {
        public void map(LongWritable key, Text salesrecord, Context con)
        throws IOException, InterruptedException {
            String[] word = salesrecord.toString().split(",");
            String country = word[7];
            try {
                Float price = Float.parseFloat(word[2]);
                con.write(new Text(country), new FloatWritable(price));
 

In [ ]:
!cat /content/Programs/reducer2.java

package com.average.wc;
import java.io.IOException;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
public class Average_reducer{
    public static class ReducerClass extends
    Reducer<Text, FloatWritable, Text, Text> {
        public void reduce(Text key, Iterable<FloatWritable> valueList,
        Context con) throws IOException, InterruptedException {
            try {
                Float total = (float) 0;
                int count = 0;
                for (FloatWritable var : valueList) {
                    total += var.get();
                    System.out.println("reducer " + var

In [ ]:
!cat /content/Programs/driver2.java

package com.avgsales.wc;
import java.io.IOException;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.FloatWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
public class Averagemain {
    public static void main(String[] args) {
        Configuration conf = new Configuration();
        try {
            Job job = Job.getInstance(conf, "FindAverageOfPrice");
            job.setJarByClass(Average_sales.class);
            job.setMapperClass(MapperClass.class);
            job.setReducerClass(ReducerClass.class);
            job.setOutputKeyClass(Text.class);
            job.setOutputValueClass(FloatWritable.class);
            Path p1 = new Path(a

### Step 3: Execution

In [ ]:
!/usr/local/hadoop-3.3.4/bin/hadoop jar /content/AvgSales.jar /content/input/SalesJan2009.csv /content/output2

2022-10-11 19:27:59,229 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-11 19:27:59,412 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-11 19:27:59,412 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-11 19:27:59,539 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2022-10-11 19:27:59,605 INFO input.FileInputFormat: Total input files to process : 1
2022-10-11 19:27:59,641 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-11 19:27:59,917 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local23432650_0001
2022-10-11 19:27:59,918 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-11 19:28:00,126 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2022-10-11 19:28:00,127 INFO mapreduce.Job: Running job: job_local23432650_0001

### Output

In [ ]:
!cat /content/output2/part-r-00000

Argentina	Average: 1200.0
Australia	Average: 1705.2632
Austria	Average: 1542.8572
Bahrain	Average: 1200.0
Belgium	Average: 1500.0
Bermuda	Average: 1200.0
Brazil	Average: 2460.0
Bulgaria	Average: 1200.0
Canada	Average: 1642.1052
Cayman Isls	Average: 1200.0
China	Average: 1200.0
Costa Rica	Average: 1200.0
Czech Republic	Average: 2000.0
Denmark	Average: 1200.0
Dominican Republic	Average: 1200.0
Finland	Average: 1200.0
France	Average: 1966.6666
Germany	Average: 1680.0
Greece	Average: 1200.0
Guatemala	Average: 1200.0
Hong Kong	Average: 1200.0
Hungary	Average: 1200.0
Iceland	Average: 1200.0
India	Average: 1200.0
Ireland	Average: 1426.5306
Israel	Average: 1200.0
Italy	Average: 2520.0
Japan	Average: 1200.0
Jersey	Average: 1200.0
Kuwait	Average: 1200.0
Latvia	Average: 1200.0
Luxembourg	Average: 1200.0
Malaysia	Average: 1200.0
Malta	Average: 2400.0
Mauritius	Average: 3600.0
Moldova	Average: 1200.0
Monaco	Average: 1200.0
Netherlands	Average: 2031.8182
New Zealand	Average: 1200.0
Norway	Average: 1